# Descargar Librerias

In [ ]:
#Descargar siempre cuando usen Colab
#!pip install praw tqdm

# Iniciar el Extractor y Completar debajo

In [1]:
import praw
import pandas as pd
import os
import time
from tqdm import tqdm

# Configura tu instancia de PRAW con tus credenciales de la aplicación y un user_agent único
reddit_config = {
    'client_id': '9zK4SSfBSCMJPnpX1rbg4Q',
    'client_secret': 'CI5_-mXS5NGVZMvNjmOb6kyE60fG_A',
    'user_agent': 'MiScriptRedditBot/v1.7 (by Sheincito en Reddit)'
}

# Define el subreddit a extraer
subreddit_to_extract = input("Ingrese el nombre del subreddit: ")

# Define el filtro para los títulos de las publicaciones
filtro_titulo = input("Ingrese el filtro para los títulos de las publicaciones (deje en blanco para no filtrar por título): ")

# Define el filtro para el cuerpo de las publicaciones
filtro_cuerpo = input("Ingrese el filtro para el cuerpo de las publicaciones (deje en blanco para no filtrar por cuerpo): ")

# Nombre del archivo CSV basado en el subreddit extraído
csv_filename = f"extractor_{subreddit_to_extract}.csv"

# Función para cargar los títulos existentes del archivo CSV
def cargar_titulos_existentes(filename, subreddit_name):
    if os.path.isfile(filename):  
        try:
            df = pd.read_csv(filename)
            if df.empty:
                return df
            else:
                return set(df['Reddit'])
        except pd.errors.EmptyDataError:
            return df
    else: 
        return pd.DataFrame(columns=[subreddit_name])

# Función para recopilar publicaciones
def recopilar_publicaciones(subreddit_name, filename, reddit, filtro_titulo=None, filtro_cuerpo=None):
    # Cargar los títulos existentes del archivo CSV
    titulos_existentes = cargar_titulos_existentes(filename, subreddit_name)
    
    # Obtiene el subreddit
    subreddit = reddit.subreddit(subreddit_name)
    
    # Lista para almacenar las publicaciones
    posts = []
    
    # Convertir el filtro de título a minúsculas si está presente
    filtro_titulo = filtro_titulo.lower() if filtro_titulo else None
    
    # Convertir el filtro de cuerpo a minúsculas si está presente
    filtro_cuerpo = filtro_cuerpo.lower() if filtro_cuerpo else None
    
    # Dividir el filtro de título en palabras si contiene varias palabras
    filtro_titulo_palabras = filtro_titulo.split() if filtro_titulo else None
    
    # Dividir el filtro de cuerpo en palabras si contiene varias palabras
    filtro_cuerpo_palabras = filtro_cuerpo.split() if filtro_cuerpo else None
    
    # Bucle para recopilar publicaciones
    for submission in tqdm(subreddit.new(limit=None), desc='Recopilando publicaciones'):
        # Convertir el título y el cuerpo a minúsculas para comparación sin distinción de mayúsculas y minúsculas
        titulo_lower = submission.title.lower()
        cuerpo_lower = submission.selftext.lower()
        
        # Verificar si la publicación cumple con al menos uno de los filtros
        if (not filtro_titulo or any(word in titulo_lower for word in filtro_titulo_palabras)) \
            or (not filtro_cuerpo or any(word in cuerpo_lower for word in filtro_cuerpo_palabras)):
            
            # Verifica si el título de la publicación ya está en los títulos existentes
            if submission.title not in titulos_existentes:
                # Agrega el título a la lista de títulos y al conjunto de títulos existentes
                titulos_existentes.add(submission.title)

                # Agrega el título y el cuerpo de la publicación a la lista de publicaciones
                posts.append({'Reddit': submission.title, 'Body': submission.selftext})

                # Espera 2 segundos antes de procesar la siguiente publicación
                time.sleep(2)
    
    # Convierte la lista de publicaciones en un DataFrame
    df = pd.DataFrame(posts)
    
    # Guarda las publicaciones en un archivo CSV (modo 'a' para agregar datos sin sobrescribir)
    df.to_csv(filename, index=False, mode='a', header=not os.path.exists(filename))
    
    # Mensaje de finalización
    print(f"Se han extraído y guardado exitosamente {len(posts)} publicaciones nuevas del subreddit '{subreddit_name}'.")
    print("Proceso finalizado.")

# Configura PRAW con la configuración especificada
reddit = praw.Reddit(**reddit_config)

# Llama a la función para recopilar publicaciones y guardarlas en un archivo CSV
recopilar_publicaciones(subreddit_to_extract, csv_filename, reddit, filtro_titulo, filtro_cuerpo)


Recopilando publicaciones: 968it [28:39,  1.78s/it]

Se han extraído y guardado exitosamente 851 publicaciones nuevas del subreddit 'argentina'.
Proceso finalizado.


# Cargamos el DF generado para probar

In [2]:
# Carga el archivo CSV que contiene los datos del DataFrame
df = pd.read_csv(csv_filename)
pd.options.display.max_colwidth = 10000

## Aqui borre los nulos por que cuando hay imagenes no las guarda y queda como NaN en el Body

In [3]:
df2=df.dropna(subset=['Body'])

In [4]:
len(df2)

677

In [5]:
len(df2['Reddit'].unique())

237

In [7]:
df2.head()

,Reddit,Body
54,Oficial: Derogacion inmediata del articulo que sube sueldos a Presidencia y compañia. Milei se baja el sueldo.,Fuente: https://twitter.com/madorni/status/1766652989341610397?t=qSVxwOcYfV_7doV-PozJGg&s=19
64,Oficial: Derogacion inmediata del articulo que sube sueldos a Presidencia y compañia. Milei se baja el sueldo.,Fuente: https://twitter.com/madorni/status/1766652989341610397?t=qSVxwOcYfV_7doV-PozJGg&s=19
75,Oficial: Derogacion inmediata del articulo que sube sueldos a Presidencia y compañia. Milei se baja el sueldo.,Fuente: https://twitter.com/madorni/status/1766652989341610397?t=qSVxwOcYfV_7doV-PozJGg&s=19
87,Oficial: Derogacion inmediata del articulo que sube sueldos a Presidencia y compañia. Milei se baja el sueldo.,Fuente: https://twitter.com/madorni/status/1766652989341610397?t=qSVxwOcYfV_7doV-PozJGg&s=19
100,Oficial: Derogacion inmediata del articulo que sube sueldos a Presidencia y compañia. Milei se baja el sueldo.,Fuente: https://twitter.com/madorni/status/1766652989341610397?t=qSVxwOcYfV_7doV-PozJGg&s=19


In [8]:
# Crear una máscara booleana para las filas que contienen la palabra "milei" en el título o el texto
mask = df2['Reddit'].str.contains('milei', case=False) | df2['Body'].str.contains('milei', case=False)

# Aplicar la máscara para filtrar las filas
df_f = df2[mask]

df_f.shape

(457, 2)

In [12]:
# Crear una máscara booleana para las filas que contienen la palabra "milei" en el título o el texto
mask = ~(df2['Reddit'].str.contains('milei', case=False) | df2['Body'].str.contains('milei', case=False))

# Aplicar la máscara para filtrar las filas
df_p = df2[mask]

df_p.shape

(220, 2)

In [13]:
df_p.head()

,Reddit,Body
6115,Virgen a los 40...,"Tengo 25, nunca he tenido pareja/novia nada serio . Estoy pensando en desvivirme. (Y no . No soy virgen de sexo. Ya me cansé del amor pago) busco algo más quiero una compañera"
6121,Amenazaron en vivo a Nelson Castro en Rosario mientras cubría nota en TN,Estaba hablando y justo le enviaron un mensaje amenazandolo de muerte
6127,"Vivo en Rosario, hagan sus preguntas...","Bueno , veo que muchos tienen dudas de como es vivir en Rosario."
6128,Lunes Random,Cosas Random
6129,"Thread Diario de Dudas, Consultas y Mitaps - 11/03",Thread Diario de Dudas y Consultas!\n\nEntra a nuestro [Discord](https://discord.gg/PeaSPMPn46) y charla con la comunidad!


In [9]:
len(df_f['Reddit'].unique())

17